In [1]:
import pandas as pd
import re
import numpy as np
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
#import torch
import json
import aiohttp
from dotenv import load_dotenv
import asyncio
import openpyxl
#from vllm import LLM, SamplingParams 


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [6]:
CATALOG_ID = 'b1gdteprkjuca0tqp6ve'
API_TOKEN = 'AQVNxbWfUTbQe6RlnFXG2k13is-KCJpygYDZh6Qx'

async def process_agronomist_message(raw_message):
    system_prompt = """Ты являешься помощником для занесения данных о выполненных сельскохозяйственных работах в таблицу."""
    
    user_prompt = f"""
    <Инструкция> 
    Сложность состоит в том, что 
    в своих отчетах агрономы используют различные сокращения, а в итоговой таблице необходимо указать полное название работы. 
    Чтобы ты смог выполнить эту задачу, я дам тебе справочную информацию (полные названия работ и всех наименовай, которые встретятся в отчетах сотрудников). Она будет приведена ниже. 
    В итоговой таблице 6 обязательных столбцов, соответственно, в отчете сотрудников должны присутствовать как минимум 6 наименований. Отдельно отмечу, что в сообщении чаще всего может быть сразу несколько описаний работ, 
    то есть агроном мог выполнять несколько работ за день и описать их в одном сообщении. Обычно в таком случае эти описания разделены пустыми строками, но могут и не быть. Если описаний несколько, то дополнительно сообщие мне об этом и укажи точное количество. 
    В этом случае ты должен будешь указать все 6 обязательных полей для каждой работы.
    Как я уже сказал, обязательных полей всего 6, но всего их 8. 


    Далее ниже будут приведены 6 обязательных полей, значения для которых 
    ОБЯЗАТЕЛЬНО должны присутствовать в таблице. Затем для первых трех полей (Подразделение, Операция, Культура) будет приведена справочная информация. В конце инструкции будут приведены два примера входных сообщений от агронома (в формате JSON) и пример вывода в итоговую таблицу. 

     6 обязательных полей: 
    'Дата' -- Дата выполнения работы (в формате MM-DD). Извлекается из сообщения агронома, если в нем есть. Если нет, то извлекается из поля 'date' (в формате YYYY-MM-DD-Time-zone) и выводится в таблице.
    'Подразделение' -- Принадлежность отделений и производственных участков (ПУ) к подразделениям
    'Операция' -- Наименование выполняемой работы 
    'Культура' -- Наименование культуры, на которой выполнялась работа 
    'За день га' -- Площадь, на которой выполнялась работа (в гектарах)
    'C начала операции' -- Площадь, на которой выполнялась работа (в гектарах) с начала операции

    Необязательные поля (информации о них может не быть в сообщении агронома): 
 
    'Вал за день, ц' -- количество собранной сельскохозяйственной культуры за день (в центнерах)
    'Вал с начала, ц' -- количество собранной сельскохозяйственной культуры за весь период (в центнерах)

    Если нет значений для полей 'Вал за день, ц' и 'Вал с начала, ц', то оставляй их пустыми. Если они есть, то вноси их в таблицу и ОБЯЗАТЕЛЬНО ставь запятую перед двумя последними цифрами, так как агрономы присылают эти значения в кг, а нужно выводить в центнерах.
    Вносить их нужно ТОЛЬКО в том случае, если они присутствуют в сообщении агронома.

    Справочная информация: 

    1. Подразделение

    | Подразделение       | Производственный участок (ПУ) | № Отделения     |
|---------------------|-------------------------------|------------------|
| АОР                 | Кавказ                        | 18               |
| АОР                 | Кавказ                        | 19               |
| АОР                 | Север                         | 3                |
| АОР                 | Север                         | 7                |
| АОР                 | Север                         | 10               |
| АОР                 | Север                         | 20               |
| АОР                 | Центр                         | 1                |
| АОР                 | Центр                         | 4                |
| АОР                 | Центр                         | 5                |
| АОР                 | Центр                         | 6                |
| АОР                 | Центр                         | 9                |
| АОР                 | Юг                            | 11               |
| АОР                 | Юг                            | 12               |
| АОР                 | Юг                            | 16               |
| АОР                 | Юг                            | 17               |
| ТСК                 | Нет ПУ                        | Нет отделения    |
| АО Кропоткинское    | Нет ПУ                        | Нет отделения    |
| Восход              | Нет ПУ                        | Нет отделения    |
| Колхоз Прогресс     | Нет ПУ                        | Нет отделения    |
| Мир                 | Нет ПУ                        | Нет отделения    |
| СП Коломейцево      | Нет ПУ                        | Нет отделения    |

2. Операция 

| Наименования полевых работ       | Примечание                                         |
|----------------------------------|----------------------------------------------------|
| 1-я междурядная культивация      | На всех культурах кроме пшеницы, ячменя           |
| 2-я междурядная культивация      | На всех культурах кроме пшеницы, ячменя           |
| Боронование довсходовое          |                                                    |
| Внесение минеральных удобрений   |                                                    |
| Выравнивание зяби                |                                                    |
| 2-е Выравнивание зяби            |                                                    |
| Гербицидная обработка            | На свекле их 4 шт, на остальных культурах 1       |
| 1 Гербицидная обработка          |                                                    |
| 2 Гербицидная обработка          |                                                    |
| 3 Гербицидная обработка          |                                                    |
| 4 Гербицидная обработка          |                                                    |
| Дискование                       |                                                    |
| Дискование 2-е                   |                                                    |
| Инсектицидная обработка          |                                                    |
| Культивация                      |                                                    |
| Пахота                           |                                                    |
| Подкормка                        |                                                    |
| Предпосевная культивация         |                                                    |
| Прикатывание посевов             |                                                    |
| Сев                              |                                                    |
| Сплошная культивация             |                                                    |
| Уборка                           |                                                    |
| Функицидная обработка            |                                                    |
| Чизлевание                       |                                                    |

3. Культура

| Наименования с/х культур               |
|----------------------------------------|
| Вика+Тритикале                         |
| Горох на зерно                         |
| Горох товарный                         |
| Гуар                                   |
| Конопля                                |
| Кориандр                               |
| Кукуруза кормовая                      |
| Кукуруза семенная                      |
| Кукуруза товарная                      |
| Люцерна                                |
| Многолетние злаковые травы            |
| Многолетние травы прошлых лет         |
| Многолетние травы текущего года       |
| Овес                                   |
| Подсолнечник кондитерский             |
| Подсолнечник семенной                 |
| Подсолнечник товарный                 |
| Просо                                  |
| Пшеница озимая на зеленый корм        |
| Пшеница озимая семенная               |
| Пшеница озимая товарная               |
| Рапс озимый                            |
| Рапс яровой                            |
| Свекла сахарная                        |
| Сорго                                  |
| Сорго кормовой                         |
| Сорго-суданковый гибрид               |
| Соя семенная                           |
| Соя товарная                           |
| Чистый пар                             |
| Чумиза                                 |
| Ячмень озимый                          |
| Ячмень озимый семенной                |

Первый пример входного сообщения от агронома: 


    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender": 
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    
    "reply_to_message_id": null,
    "raw_text": "Пахота зяби под мн тр
По Пу 26/488
Отд 12 26/221

Предп культ под оз пш
По Пу 215/1015
Отд 12 128/317
Отд 16 123/529

2-е диск сах св под пш
По Пу 22/627
Отд 11 22/217

2-е диск сои под оз пш
По Пу 45/1907
Отд 12 45/299"


Пример вывода: 

| Подразделение | Вид работ                | С/х культура                   | № поля | Площадь (га) |
|---------------|--------------------------|--------------------------------|--------|--------------|
| АОР           | Пахота                   | Многолетние травы текущего года | 26     | 488          |
| АОР           | Предпосевная культивация | Пшеница озимая товарная        | 215    | 1015         |
| АОР           | Дискование 2-е           | Пшеница озимая товарная        | 22     | 627          |
| АОР           | Дискование 2-е           | Пшеница озимая товарная        | 45     | 1907         |

Отдельно замечу, что если в сообщении есть фраза 'По ПУ' или 'По Пу', то в поля '№ поля' и 'Площадь (га)' необходимо вносить значения, которые следуют после этой фразы.


Второй пример входного сообщения от агронома: 


    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender": 
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    ,
    "reply_to_message_id": null,
    "raw_text": "Уборка свеклы 27.10.день
Отд10-45/216
По ПУ 45/1569
Вал 1259680/6660630
Урожайность 279,9/308,3
По ПУ 1259680/41630600
На завод 1811630/6430580
По ПУ 1811630/41400550
Положено в кагат 399400
Вввезено с кагата 951340
Остаток 230060
Оз-9,04/12,58
Дигестия-14,50/15,05"



Таблица: 

| Дата       | Подразделение | Операция | Культура         | За день, га | С начала операции, га | Вал за день, ц | Вал с начала, ц |
|------------|----------------|----------|------------------|-------------|------------------------|----------------|------------------|
| 10/27/2024 | АОР            | Уборка   | Свекла сахарная  | 45          | 1 569                  | 12 596,80      | 66 606,30        |


    Проанализируй следующее сообщение от агронома и преобразуй его в структурированную таблицу в соответствии с приведенной выше инструкцией. 
    Учти, что из сообщения можно извлечь только 5 обязательных полей, а остальные поля могут отсутствовать. Но при этом если есть все 8 полей, 
    то они тоже должны быть указаны в таблице. Поле 'Дата' ты либо извлекаешь из сообщения, либо берешь из поля 'date' (в формате YYYY-MM-DD-Time-zone).

    {raw_message}
    
    Верни ответ в виде таблицы в формате markdown, где каждая строка представляет одну операцию. Укажи количество найденных операций. В конце ОБЯЗАТЕЛЬНО проверь,
    что все занесенные тобой в таблицу данные корректны и соответствуют наименованиям из справочной информации. Если не соответсвуют, то исправь. 
    </Инструкция>
    """
    
    try:
        result = await call_yandex_gpt_async(user_prompt, system_prompt)
        return result
    except Exception as e:
        return {"error": str(e)}

# Модифицируем функцию API для возврата результата
async def call_yandex_gpt_async(user_prompt, system_prompt=None):
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_TOKEN}"
    }
    payload = {
        "modelUri": f"gpt://{CATALOG_ID}/yandexgpt/rc",
        "completionOptions": {"maxTokens": 3000, "temperature": 0.3}, 
        "messages": [
            {"role": "system", "text": system_prompt if system_prompt else "Ошибка"},
            {"role": "user", "text": user_prompt}
        ]
    }

    async with aiohttp.ClientSession() as session:
        async with session.post(url, headers=headers, json=payload) as response:
            if response.status == 200:
                result = await response.json()
                return result.get("result", {}).get("alternatives", [{}])[0].get("message", {}).get("text", "")
            else:
                error_text = await response.text()
                return f"Ошибка API: {response.status} - {error_text}"

In [ ]:
async def main():
    test_message = """{
    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender": {
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    },
    "reply_to_message_id": null,
    "raw_text": "Пахота зяби под сою 
По ПУ 7/1402
Отд 17 7/141

Вырав-ие зяби под кук/силос
По ПУ 16/16
Отд 12 16/16

Вырав-ие зяби под сах/свёклу
По ПУ 67/912
Отд 12 67/376

2-ое диск-ие сах/свёкла 
По ПУ 59/1041
Отд 17 59/349"
}"""

    result = await process_agronomist_message(test_message)
    
    print("\n Итоговая таблица: \n")
    print(result)
    
    # Место для кода для сохранения результата в Excel таблицу, пока не реализовано
    
if __name__ == "__main__":
    await main()


 Итоговая таблица: 

| Дата              | Подразделение | Операция                   | Культура             | За день, га | С начала операции, га |
|-------------------|---------------|----------------------------|----------------------|-------------|------------------------|
| 2025-04-13        | АОР           | Пахота                     | Соя                  | 7           | 1402                   |
| 2025-04-13        | АОР           | Выравнивание зяби          | Кукуруза на силос    | 16          | 16                     |
| 2025-04-13        | АОР           | Выравнивание зяби          | Свекла сахарная      | 67          | 912                    |
| 2025-04-13        | АОР           | Дискование 2-е             | Свекла сахарная      | 59          | 1041                   |

Количество найденных операций: 4


In [22]:
async def main():
    test_message = """{
    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender": {
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    },
    "reply_to_message_id": null,
    "raw_text": "Пахота под сах св
По Пу 77/518
Отд 12 46/298
Отд 16 21/143
Отд 17 10/17

Чизел под оз ячмень 
По Пу 22/640
Отд 11 22/242

Чизел под оз зел корм
Отд 11 40/40

Диск оз пшеницы
По Пу 28/8872
Отд 17 28/2097

2-е диск под сах св
По Пу 189/1763
Отд 11 60/209
Отд 12 122/540
Отд 17 7/172

Диск кук силос
По Пу 6/904
Отд 11 6/229

Прик под оз ячмень
По Пу 40/498
Отд 11 40/100

Уборка сои (семенной)
Отд 11 65/65
Вал 58720
Урож 9"
}"""

    result = await process_agronomist_message(test_message)
    
    print("\n Итоговая таблица: \n")
    print(result)
    
    # Место для кода для сохранения результата в Excel таблицу
    
if __name__ == "__main__":
    await main()


 Итоговая таблица: 

| Дата       | Подразделение | Операция                  | Культура                   | За день, га | С начала операции, га | Вал за день, ц | Вал с начала, ц |
|------------|----------------|---------------------------|----------------------------|-------------|------------------------|----------------|------------------|
| 04-13-2025 | АОР            | Пахота                    | Свекла сахарная            | 77          | 518                    |                |                  |
| 04-13-2025 | АОР            | Чизлевание                | Ячмень озимый              | 22          | 640                    |                |                  |
| 04-13-2025 | АОР            | Чизлевание                | Пшеница озимая на зеленый корм | 40          | 40                     |                |                  |
| 04-13-2025 | АОР            | Дискование                | Пшеница озимая товарная    | 28          | 8872                   |                |             

In [31]:
async def main():
    test_message = """{
    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender": {
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    },
    "reply_to_message_id": null,
    "raw_text": "Пахота под сах св
ПоПу 91/609
Отд 11 13/73
Отд 12 49/347
Отд 16 20/163
Отд 17 9/26

Чизел под оз зел корм
Отд 11 60/100

2-е диск под сах св
По Пу 53/1816
Отд 12 53/593

Диск кук силос
По Пу 66/970
Отд 11 66/295

Диск сах св
Отд 12 13/13

Прикат под оз ячмень
По Пу 40/538
Отд 11 40/140

Уборка сои семенной
Отд 11 29/94
Вал 37400/96120
Урож 12,9/10,2"
}"""
   


    result = await process_agronomist_message(test_message)
    
    print("\n Итоговая таблица: \n")
    print(result)
    

    # Место для кода для сохранения результата в Excel таблицу

    lines = result.strip().split('\n')

    headers = [col.strip() for col in lines[0].split('|')[1:-1]]

    data = []
    for line in lines[2:]:
        row = [cell.strip() for cell in line.split('|')[1:-1]]
        data.append(row)
    new_df = pd.DataFrame(data, columns=headers)
    
    output_file = 'table_output.xlsx'
    
    try:
        existing_df = pd.read_excel(output_file)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    except FileNotFoundError:
        combined_df = new_df
    combined_df.to_excel(output_file, index=False)

    

    
if __name__ == "__main__":
    await main()


 Итоговая таблица: 

| Дата       | Подразделение | Операция                    | Культура                   | За день, га | С начала операции, га | Вал за день, ц | Вал с начала, ц |
|------------|----------------|-----------------------------|----------------------------|-------------|------------------------|----------------|------------------|
| 04-13-2025 | АОР            | Пахота                      | Свекла сахарная            | 91          | 609                    |                |                  |
| 04-13-2025 | АОР            | Чизлевание                  | Пшеница озимая на зеленый корм | 60          | 100                    |                |                  |
| 04-13-2025 | АОР            | Дискование 2-е              | Свекла сахарная            | 53          | 1816                   |                |                  |
| 04-13-2025 | АОР            | Дискование                  | Кукуруза кормовая          | 66          | 970                    |                | 

In [3]:
async def main():
    test_message = """{
    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender": {
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    },
    "reply_to_message_id": null,
    "raw_text": "Восход
Посев кук-24/252га
24%
Предпосевная культ
Под кук-94/490га46%
СЗРоз пш-103/557га
25%
Подкормка оз рапс-
152га , 100%, подкормка овса-97га, 50%
Довсходовое боронование подсолнечника-524
га, 100%."
}"""
   


    result = await process_agronomist_message(test_message)
    
    print("\n Итоговая таблица: \n")
    print(result)
    

    # Место для кода для сохранения результата в Excel таблицу

    lines = result.strip().split('\n')

    headers = [col.strip() for col in lines[0].split('|')[1:-1]]

    data = []
    for line in lines[2:]:
        row = [cell.strip() for cell in line.split('|')[1:-1]]
        data.append(row)
    new_df = pd.DataFrame(data, columns=headers)
    
    output_file = 'table_output.xlsx'
    
    try:
        existing_df = pd.read_excel(output_file)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    except FileNotFoundError:
        combined_df = new_df
    combined_df.to_excel(output_file, index=False)

    

    
if __name__ == "__main__":
    await main()


 Итоговая таблица: 

| Дата       | Подразделение | Операция                  | Культура                   | За день, га | С начала операции, га | Вал за день, ц | Вал с начала, ц |
|------------|----------------|---------------------------|----------------------------|-------------|------------------------|----------------|------------------|
| 04/13/2025 | Восход         | Посев                     | Кукуруза товарная          | 24          | 252                    |                |                  |
| 04/13/2025 | Восход         | Предпосевная культивация  | Кукуруза товарная          | 94          | 490                    |                |                  |
| 04/13/2025 | Восход         | Сев                       | Пшеница озимая товарная    | 103         | 557                    |                |                  |
| 04/13/2025 | Восход         | Подкормка                 | Рапс озимый                | 152         |                        |                |                 

В данном случае, ошибка, не хватает операции "Гербицидная обработка".

In [5]:
async def main():
    test_message = """{
    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender": {
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    },
    "reply_to_message_id": null,
    "raw_text": "18.11 Мир
Пахота зяби под кукурузу 25 га день, 664 га от начала, 99%, 8 га остаток.
Пахота зяби под сою 40 га день, 939 га от начала, 83%, 200 га остаток.
Работало 3 агрегата.
Выравнивание зяби под сахарную свёклу 45 га день, 952 га от начала, 100%. Работал 1 агрегат."
}"""
   


    result = await process_agronomist_message(test_message)
    
    print("\n Итоговая таблица: \n")
    print(result)
    

    # Место для кода для сохранения результата в Excel таблицу

    lines = result.strip().split('\n')


    headers = [col.strip() for col in lines[0].split('|')[1:-1]]

    data = []
    for line in lines[2:]:
        row = [cell.strip() for cell in line.split('|')[1:-1]]
        data.append(row)
    new_df = pd.DataFrame(data, columns=headers)
    
    output_file = 'table_output.xlsx'
    
    try:
        existing_df = pd.read_excel(output_file)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    except FileNotFoundError:
        combined_df = new_df
    combined_df.to_excel(output_file, index=False)

    

    
if __name__ == "__main__":
    await main()


 Итоговая таблица: 

| Дата       | Подразделение | Операция                  | Культура            | За день, га | С начала операции, га |
|------------|----------------|---------------------------|---------------------|------------|-----------------------|
| 11/18/2024 | Мир            | Пахота                   | Кукуруза            | 25         | 664                    |
| 11/18/2024 | Мир            | Пахота                   | Соя                 | 40         | 939                    |
| 11/18/2024 | Мир            | Выравнивание зяби        | Свекла сахарная     | 45         | 952                    |

Количество найденных операций: 3


Выравн под сою
ПоПу 53/1441
Отд 11 23/672
Отд 12 30/251
Выравн под кук силос
ПоПу 6/871
Отд 11 6/401
Предп культ под сах св
ПоПу 22/406
Отд 11 20/67
Отд 12 2/106
Сев сах св
ПоПу 15/122
Отд 11 5/5
Отд 16 10/88"

In [4]:
async def main():
    test_message = """{
    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender": {
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    },
    "reply_to_message_id": null,
    "raw_text": "Выравн под сою
ПоПу 53/1441
Отд 11 23/672
Отд 12 30/251
Выравн под кук силос
ПоПу 6/871
Отд 11 6/401
Предп культ под сах св
ПоПу 22/406
Отд 11 20/67
Отд 12 2/106
Сев сах св
ПоПу 15/122
Отд 11 5/5
Отд 16 10/88"
}"""
   


    result = await process_agronomist_message(test_message)
    
    print("\n Итоговая таблица: \n")
    print(result)
    

    # Место для кода для сохранения результата в Excel таблицу

    lines = result.strip().split('\n')


    headers = [col.strip() for col in lines[0].split('|')[1:-1]]

    data = []
    for line in lines[2:]:
        row = [cell.strip() for cell in line.split('|')[1:-1]]
        data.append(row)
    new_df = pd.DataFrame(data, columns=headers)
    
    output_file = 'table_output.xlsx'
    
    try:
        existing_df = pd.read_excel(output_file)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    except FileNotFoundError:
        combined_df = new_df
    combined_df.to_excel(output_file, index=False)

    

    
if __name__ == "__main__":
    await main()


 Итоговая таблица: 

| Дата       | Подразделение | Операция                    | Культура                | За день, га | С начала операции, га |
|------------|----------------|-----------------------------|-------------------------|-------------|------------------------|
| 04/13/2025 | АОР            | Выравнивание зяби           | Соя товарная            | 53          | 1441                   |
| 04/13/2025 | АОР            | Выравнивание зяби           | Кукуруза кормовая       | 6           | 871                    |
| 04/13/2025 | АОР            | Предпосевная культивация    | Свекла сахарная         | 22          | 406                    |
| 04/13/2025 | АОР            | Сев                         | Свекла сахарная         | 15          | 122                    |

Количество найденных операций: 4.


In [4]:
async def main():
    test_message = """{
    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender": {
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    },
    "reply_to_message_id": null,
    "raw_text": "12.10
Внесение мин удобрений под оз пшеницу 2025 г ПУ Юг 149/7264
Отд 17-149/1443"
}"""
   


    result = await process_agronomist_message(test_message)
    
    print("\n Итоговая таблица: \n")
    print(result)
    

    # Место для кода для сохранения результата в Excel таблицу

    lines = result.strip().split('\n')


    headers = [col.strip() for col in lines[0].split('|')[1:-1]]

    data = []
    for line in lines[2:]:
        row = [cell.strip() for cell in line.split('|')[1:-1]]
        data.append(row)
    new_df = pd.DataFrame(data, columns=headers)
    
    output_file = 'table_output.xlsx'
    
    try:
        existing_df = pd.read_excel(output_file)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    except FileNotFoundError:
        combined_df = new_df
    combined_df.to_excel(output_file, index=False)

    
if __name__ == "__main__":
    await main()


 Итоговая таблица: 

| Дата       | Подразделение | Операция                      | Культура                  | За день, га | С начала операции, га |
|------------|----------------|--------------------------------|---------------------------|-------------|------------------------|
| 10/12/2025 | АОР            | Внесение минеральных удобрений | Пшеница озимая товарная   | 149         | 7264                   |

Количество найденных операций: 1.


In [7]:
async def main():
    test_message = """{
    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender": {
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    },
    "reply_to_message_id": null,
    "raw_text": "Пахота под сах св
По Пу 88/329
Отд 11 23/60
Отд 12 34/204
Отд 16 31/65

Пахота под мн тр
По Пу 10/438
Отд 17 10/80

Чизел под оз ячмень
По Пу 71/528
Отд 11 71/130

2-е диск под сах св
По Пу 80/1263
Отд 12 80/314

2-е диск под оз ячмень
По Пу 97/819
Отд 17 97/179

Диск кук силос
По Пу 43/650
Отд 11 33/133
Отд 12 10/148

Выкаш отц форм под/г
Отд 12 10/22

Уборка сах св
Отд 12 16/16
Вал 473920
Урож 296,2
Диг - 19,19
Оз - 5,33"
}"""
   


    result = await process_agronomist_message(test_message)
    
    print("\n Итоговая таблица: \n")
    print(result)
    

    # Место для кода для сохранения результата в Excel таблицу

    lines = result.strip().split('\n')


    headers = [col.strip() for col in lines[0].split('|')[1:-1]]

    data = []
    for line in lines[2:]:
        row = [cell.strip() for cell in line.split('|')[1:-1]]
        data.append(row)
    new_df = pd.DataFrame(data, columns=headers)
    
    output_file = 'table_output.xlsx'
    
    try:
        existing_df = pd.read_excel(output_file)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    except FileNotFoundError:
        combined_df = new_df
    combined_df.to_excel(output_file, index=False)

    
if __name__ == "__main__":
    await main()


 Итоговая таблица: 

| Дата       | Подразделение | Операция                  | Культура                   | За день, га | С начала операции, га | Вал за день, ц | Вал с начала, ц |
|------------|----------------|---------------------------|----------------------------|-------------|------------------------|----------------|------------------|
| 04/13/2025 | АОР            | Пахота                    | Свекла сахарная            | 88          | 329                    |                |                  |
| 04/13/2025 | АОР            | Пахота                    | Многолетние травы текущего года | 10         | 438                    |                |                  |
| 04/13/2025 | АОР            | Чизлевание                 | Ячмень озимый              | 71          | 528                    |                |                  |
| 04/13/2025 | АОР            | Дискование 2-е             | Свекла сахарная            | 80          | 1263                   |                |           

In [8]:
async def main():
    test_message = """{
    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender": {
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    },
    "reply_to_message_id": null,
    "raw_text": "Восход
Посев кук-24/252га
24%
Предпосевная культ
Под кук-94/490га46%
СЗРоз пш-103/557га
25%
Подкормка оз рапс-
152га , 100%, подкормка овса-97га, 50%
Довсходовое боронование подсолнечника-524
га, 100%."
}"""
   


    result = await process_agronomist_message(test_message)
    
    print("\n Итоговая таблица: \n")
    print(result)
    

    # Место для кода для сохранения результата в Excel таблицу

    lines = result.strip().split('\n')


    headers = [col.strip() for col in lines[0].split('|')[1:-1]]

    data = []
    for line in lines[2:]:
        row = [cell.strip() for cell in line.split('|')[1:-1]]
        data.append(row)
    new_df = pd.DataFrame(data, columns=headers)
    
    output_file = 'table_output.xlsx'
    
    try:
        existing_df = pd.read_excel(output_file)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    except FileNotFoundError:
        combined_df = new_df
    combined_df.to_excel(output_file, index=False)

    
if __name__ == "__main__":
    await main()


 Итоговая таблица: 

| Дата       | Подразделение | Операция                  | Культура                   | За день, га | С начала операции, га | Вал за день, ц | Вал с начала, ц |
|------------|----------------|---------------------------|----------------------------|-------------|------------------------|----------------|------------------|
| 04/13/2025 | Восход          | Сев                       | Кукуруза кормовая          | 24          | 252                    |                |                  |
| 04/13/2025 | Восход          | Предпосевная культивация  | Кукуруза кормовая          | 94          | 490                    |                |                  |
| 04/13/2025 | Восход          | Подкормка                 | Рапс озимый                | 152         |                        |                |                  |
| 04/13/2025 | Восход          | Подкормка                 | Овес                       | 97          |                        |                |             